# Which free apps are the most profitable on App Store and Google Play

*This project is about analysing applications free to download and install, where main source of revenue will be from in-apps ads.  The thing is that the number of users on free apps is a key indicator of ad revenue of these apps.*

*The goal of the project is to understand which applications attract the users more in order to create a vision of those types of apps and, accordingly, target audience of those apps.*

*This data representation helps us determine key features of most popular apps, and, in addition, which of those are the most profitable ones.*

# Opening and exploring data

**At first**, we need a lot of data. As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.

Not sure we need it all. In order to save time and money on collecting data, let's take a slice of this data, that is available on the web:

* A data set containing data about approximately ten thousand Android apps from `Google Play`. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
* A data set containing data about approximately seven thousand iOS apps from the `App Store`. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).

Let's open and explore some of the data inside, AS for App Store, and GS for Google Play Store accordingly:

In [2]:
from csv import reader

### The App Store data set ###
open_apple_apps = open('AppleStore.csv')
apple_apps = list(reader(open_apple_apps))
apple_apps_header = apple_apps[0]
apple_apps = apple_apps[1:]
open_apple_apps.close()

### The Google Play Store data set ###
open_google_apps = open('googleplaystore.csv')
google_apps = list(reader(open_google_apps))
google_apps_header = google_apps[0]
google_apps = google_apps[1:]
open_google_apps.close()

**Secondly**, we need to create a function to repeatedly retrieve the information inside data sets, with opportunity to see quantity of apps and column headings:

In [3]:
def explore_data(data_set, end, start = 0, rows_and_columns=False):
    data_set_slice = data_set[start:end]
    for row in data_set_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(data_set))
        print('Number of columns:', len(data_set[0]))

**Next**, let's see what we have on App Store and Google Play Store:

In [4]:
print(apple_apps_header, '\n')
explore_data(apple_apps, 3, 0, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


Here, in `App Store`, we could see 16 columns. Most interesting for us are:

`'track_name'`: App name, **index 1**; `'price'`: Price amount, **index 4**; `'rating_count_tot'`: User Rating counts (for all version), **index 5**; `'user_rating'`: Average User Rating value (for all version); `'cont_rating'`: Content Rating; `'prime_genre'`: Primary Genre, **index 11**; `'sup_devices.num'`: Number of supporting devices, `'lang.num'`: Number of supported languages.

In [5]:
print(google_apps_header, '\n')
explore_data(google_apps, 3, 0, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


In `Google Play Store` let's highlight the next applications metrics:

`'App'`: Application name, **index 0**; `'Category'`: Category the app belongs to, **index 1**; `'Rating'`: Overall user rating of the app; `Reviews`: Number of user reviews for the app, **index 3**; `'Installs'`: Number of user downloads/installs for the app; `'Type'`: Paid or Free; `Price`: Price amount, **index 7**; `'Content Rating'`: Age group the app is targeted at - Children / Mature 21+ / Adult; `'Genres'`: An app can belong to multiple genres (apart from its main category), **index 9**.

# Cleaning the data

First step of analisys is cleaning the data.
Lets's start with `incorrect or incomplete data`.

Let's have a look at **Google Play Store** data set. From discussion section we could see that there is a problem with the row 10473 - no category of app. We need to delete it and take a look at it again.

In [6]:
print(google_apps[10472])
del google_apps[10472]
print(google_apps[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


Could not find any information about errors in App Store data set, except for `data duplication`. Let's check on this, starting from App Store data set.
Function **duplicate_check** will take care of this job.
Just for our convenience I put app name and it's index in function parametres.

In [7]:
def duplicate_check(data_set, data_set_name, app_name_index):
    print(f'Lenth of {data_set_name} is {len(data_set)}')
    duplicate_names = []
    unique_names = []
    for i in data_set:
        if (i[app_name_index] in unique_names) and (i[app_name_index] not in duplicate_names):
            duplicate_names.append(i[app_name_index])
        elif i[app_name_index] not in unique_names:
            unique_names.append(i[app_name_index])
    print(f'Quantity of duplicate names: {len(duplicate_names)}\nSome of duplicates:\n{duplicate_names[:5]}')

In [8]:
duplicate_check(apple_apps, 'App_Store', 1)
print('\n')
duplicate_check(google_apps, 'Google_Store', 0)

Lenth of App_Store is 7197
Quantity of duplicate names: 2
Some of duplicates:
['Mannequin Challenge', 'VR Roller Coaster']


Lenth of Google_Store is 10840
Quantity of duplicate names: 798
Some of duplicates:
['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


In order `not to remove the duplicates randomly` we need to get back to the core of our research.
On my opinion, to determine the most profitable apps we need to take into account 2 metrics: the number of downloads and user activity within the application (activity for a certain period).

In our data sets we have only `reviews` and its number as `significant metric of popularity` of the app.
Thus, I could suggest to remove the data lines with duplicate names which have fewer reviews.
Literally, with function **clean_duplicates** we do the following:

1. Creating a `dictionary of unique apps` with maximum reviews
2. Adding to dictionary only unique apps (names of apps) as keys
3. Adding to value the number from review column, if the previous value is lower
4. Creatind a final `list of cleaned data set` and auxiliary list (of already added apps' names) in order not to add apps with the same value more than one time
5. Adding to final list row from original (non-cleaned) data set, if apps name and its review value matches with ones in `dictionary of unique apps`

In [9]:
def clean_duplicates(data_set, name_index, review_index, duplicates_number):
    apps_max_reviews = {}
    for row in data_set:
        name = row[name_index]
        review_value = float(row[review_index])
        if name in apps_max_reviews and apps_max_reviews[name] < review_value:
            apps_max_reviews[name] = review_value
        elif name not in apps_max_reviews:
            apps_max_reviews[name] = review_value
    print(f'Quantity without duplicates must be {len(data_set) - duplicates_number}\nQuantity of unique apps is {len(apps_max_reviews)}')
          
    cleaned_data_set = []
    already_added = []
    for row in data_set:
        name = row[name_index]
        review_value = float(row[review_index])
        if (apps_max_reviews[name] == review_value) and (name not in already_added):
            cleaned_data_set.append(row)
            already_added.append(name)
    print(f'Lenth of cleaned data set is {len(cleaned_data_set)}\n')
    return cleaned_data_set

In [10]:
apple_apps = clean_duplicates(apple_apps, 1, 5, 2)
google_apps = clean_duplicates(google_apps, 0, 3, 1181)

Quantity without duplicates must be 7195
Quantity of unique apps is 7195
Lenth of cleaned data set is 7195

Quantity without duplicates must be 9659
Quantity of unique apps is 9659
Lenth of cleaned data set is 9659



Above, just as expected, there is correct lenth without duplicates for both of cleaned data sets.

Next step of cleaning will be `removing non-English data`, because we are developing apps for English-speaking audience.
Suggestion is to create a function, that will filter out the apps by its names.
Here ASCII standard could help us, it encodes alphabet and many more symbols, - the numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, including digits from 0 to 9, punctuation marks and other symbols (for ex.: +, -, /).

The thing is that name of application could contain different characters, for example like here: 'Instachat 😜' or 'Docs To Go™ Free Office Suite'. So, it is better to remove the line, if there are many of other characters outside the ASCII system.

Thus, if app's name doesn't contain more than 3 characters with ASCII corresponding number over 127 the function **check_if_english** returns True. And further, function **filter_english** will append the line of data to filtered data set.

In [11]:
def check_if_english(name):
    name_check = []
    for symbol in name:
            if ord(symbol) > 127:
                name_check.append(symbol)
            if len(name_check) > 3:
                return False
    return True
            
def filter_english(data_set, name_index):
    filtered_data_set = []
    for row in data_set:
        if check_if_english(row[name_index]):
            filtered_data_set.append(row)
    
    print(f'Lenth of data set was {len(data_set)}\nLenth of filtered data set is {len(filtered_data_set)}\n')
    return filtered_data_set

Let's check the function on our data sets and determine how many rows remain.

In [12]:
apple_apps = filter_english(apple_apps, 1)
google_apps = filter_english(google_apps, 0)
print(apple_apps[:5])
print(google_apps[:5])

Lenth of data set was 7195
Lenth of filtered data set is 6181

Lenth of data set was 9659
Lenth of filtered data set is 9614

[['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'], ['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'], ['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1'], ['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']]
[['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0

Last step for our data cleaning process will be `removing non-free apps`.
For that **filter_free** function.

In [13]:
def filter_free(data_set, price_index):
    filtered_data_set = []
    f = []
    for row in data_set:
        if row[price_index] == '0' or (row[price_index] == '0.0') or (row[price_index] == '0.00'):
            filtered_data_set.append(row)
            
    print(f'Lenth of filtered data set is {len(filtered_data_set)}\n')
    return filtered_data_set

In [14]:
apple_apps = filter_free(apple_apps, 4)
google_apps = filter_free(google_apps, 7)

Lenth of filtered data set is 3220

Lenth of filtered data set is 8864



We now have a total of 12 084 entries of data, which is enougth for our analysis.

# Research strategy

As we mentioned before, our `main goal is to find the most popular kinds of apps`, in other words which are likely to attract more users.

Considering that creating an application for both platforms is long and resource-intensive process, we will act on the basis of the following steps:

* build a minimal Android version of the app, and add it to Google Play
* if the app has a good response from users, we develop it further
* if the app is profitable after six months, we build an iOS version of the app and add it to the App Store

So for now we just need to generally measure the apps' popularity.

# Most Common Apps by Genre

Let's begin our analysis with determination of genres that are of interest to  both markets.
As we could see, Google Play Store has two suitable columns: `Category` and `Genres`, and Apple Store has `prime_genre`.

So, we need to build two functions:
* one, **freq_table**, for generating frequency table with percantages for category and its amount of users accordingly.
* and second one, **display_freq_table**, for more clarity with our data for sorting it in descending order.

In [15]:
def freq_table(data_set, genre_index):
    freq_table = {}
    for row in data_set:
        if row[genre_index] not in freq_table:
            freq_table[row[genre_index]] = 1
        else:
            freq_table[row[genre_index]] += 1
            
    for key in freq_table:
        freq_table[key] = (freq_table[key] / len(data_set)) * 100
    
    print(f'Amount of categories: {len(data_set)}\nAmount of unique categories: {len(freq_table)}\n')
    return freq_table
    
def display_freq_table(data_set, genre_index):
    table = freq_table(data_set, genre_index)
    display_freq_table = []
    for key in table:
        display_freq_table.append((table[key], key))
        
    display_freq_table = sorted(display_freq_table, reverse = True)
    print(f'Category : percantage of users\n')
    for entry in display_freq_table:
        print(f'{entry[1]} : {round(entry[0], 2)}')
    return display_freq_table

Let's examine `Google Play Store` main genre - `Category` column.

In [16]:
google_genres_major = display_freq_table(google_apps, 1)

Amount of categories: 8864
Amount of unique categories: 33

Category : percantage of users

FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


As we could see above, the most popular genre is `FAMILY` with 18.91 %.
On second place with twice less amount of apps is `GAME` (9.72 %) and near goes `TOOLS` (8.46 %).

In [17]:
google_genres_minor = display_freq_table(google_apps, 9)

Amount of categories: 8864
Amount of unique categories: 114

Category : percantage of users

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.91
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;A

Here we deduce that popular minor genre is `Tools` (8.45 %), and let's keep in mind `Entertainment` (6.07 %).
For now is seems Google Store free English apps are made more for practical purposes than for entertainment. However, if we will look for `Family` category in store, we'll see that it mostly consist of kids' games.

In [18]:
apple_genres = display_freq_table(apple_apps, 11)

Amount of categories: 3220
Amount of unique categories: 23

Category : percantage of users

Games : 58.14
Entertainment : 7.89
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.52
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.34
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


In Apple Store applications data set most common genre is `Games`, it has overwhelming majority with 58.14 % of all applications. Also we can highlight `Entertainment` with 7.89 %.
Overall impression is that most of applications focused on entertainment.

In total for now, we cannot say that building, for instance, another one game application is good idea - we need to look at the audience size of entertainment apps first.

# Apple Store genre with most users

Continuing our analisys of apps with finding genres that are the most popular among users.

Starting with `Apple Store`, since it doesn't have `installs` column, as Google Play Store has, let's take `rating_count_tot` instead.
To find average rating count (average amount of users per app) for each genre we need:
* isolate the apps of each genre
* add up the user ratings for the apps of each genre
* divide the sum by the number of each genre's apps

Let's create a function **avg_count_by_genre** (which will also sort output by users' amount for each category) and use it on Apple Store data set.

In [19]:
def avg_count_by_genre(data_set, genre_index, rating_count_index):
    unique_genres = {}
    for entry in data_set:
        genre = entry[genre_index]
        if genre in unique_genres:
            prev_sum, prev_count = unique_genres[genre]
            unique_genres[genre] = (prev_sum + int(entry[rating_count_index]), prev_count + 1)
        else:
            unique_genres[genre] = (int(entry[rating_count_index]), 1)
    
    genres_by_users = {}
    for genre in unique_genres:
        average_ratings_count = unique_genres[genre][0] / unique_genres[genre][1]
        genres_by_users[genre] = average_ratings_count
        
    genres_by_users_tuples = sorted(genres_by_users.items(), key = lambda x: x[1], reverse = True)
    genres_by_users_sorted = {k : v for k, v in genres_by_users_tuples}
    
    print(f'Category : average amount of users/installs\n')
    for entry in genres_by_users_sorted:
        print(f'{entry} : {round(genres_by_users_sorted[entry], 2)}')
    return genres_by_users_sorted

In [20]:
apple_popular_genre = avg_count_by_genre(apple_apps, 11, 5)

Category : average amount of users/installs

Navigation : 86090.33
Reference : 74942.11
Social Networking : 71548.35
Music : 57326.53
Weather : 52279.89
Book : 39758.5
Food & Drink : 33333.92
Finance : 31467.94
Photo & Video : 28441.54
Travel : 28243.8
Shopping : 26919.69
Health & Fitness : 23298.02
Sports : 23008.9
Games : 22812.92
News : 21248.02
Productivity : 21028.41
Utilities : 18684.46
Lifestyle : 16485.76
Entertainment : 14029.83
Business : 7491.12
Education : 7003.98
Catalogs : 4004.0
Medical : 612.0


We see here, that `Navigation` has the most number of users. Near goes `Reference` and `Social Networking`.
Really interesting, that there not so many apps for these genres, - we could see it from our analisys above:

Category : App's scope, in %

Social Networking : 3.29  
Reference : 0.56  
Navigation : 0.19

Amount of apps for Navigation and Reference is even less than 1 percent of the whole apps market.
Here we can assume, that Apple Store needs more of these 3 kinds of apps.
But first it is better to look at the apps with the greatest scope of users.

In [21]:
print('Navigation\n')
print(f'Name of application: its number of ratings\n')
for entry in apple_apps:
    if entry[11] == 'Navigation':
        print(f'{entry[1]} : {entry[5]}')

Navigation

Name of application: its number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


The picture with `Navigation` apps looks unrepresentative - `Waze` and `Google Maps` took the most of ratings. It cannot be reccommend to built navigation app - this will take a lot of effort and competition in this niche will be high.

In [22]:
print('Reference\n')
print(f'Name of application: its number of ratings\n')      
for entry in apple_apps:
    if entry[11] == 'Reference':
        print(f'{entry[1]} : {entry[5]}')

Reference

Name of application: its number of ratings

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


This category - `Reference`, - seems truly interesting.
The Internet has existed not so long ago, not all dictionaries, reference books, various kinds of books are in the public domain, not to mention applications for interactive work with them.

In [23]:
print('Social Networking\n')
print(f'Name of application: its number of ratings\n')
for entry in apple_apps:
    if entry[11] == 'Social Networking':
        print(f'{entry[1]} : {entry[5]}')

Social Networking

Name of application: its number of ratings

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
Gro

A similar picture was assumed in `Social Networking`, that is why we analysed this category in the second place.
Several giants are located in the first places, such as `Facebook`, `Pinterest`, `Skype for iPhone`.
It is not an easy job to create something like these social media. Furthermore, not every social application becomes super popular, along with navigation apps. Apps of these genres, that have a lot of users, are already tested over the years, and more importantly - have a huge auditory, which is must-have for social networking, and works for maps' developing too.

Nex popular app genre is `Music`.

In [24]:
print('Music\n')
print(f'Name of application: its number of ratings\n')      
for entry in apple_apps:
    if entry[11] == 'Music':
        print(f'{entry[1]} : {entry[5]}')

Music

Name of application: its number of ratings

Pandora - Music & Radio : 1126879
Spotify Music : 878563
Shazam - Discover music, artists, videos & lyrics : 402925
iHeartRadio – Free Music & Radio Stations : 293228
SoundCloud - Music & Audio : 135744
Magic Piano by Smule : 131695
Smule Sing! : 119316
TuneIn Radio - MLB NBA Audiobooks Podcasts Music : 110420
Amazon Music : 106235
SoundHound Song Search & Music Player : 82602
Sonos Controller : 48905
Bandsintown Concerts : 30845
Karaoke - Sing Karaoke, Unlimited Songs! : 28606
My Mixtapez Music : 26286
Sing Karaoke Songs Unlimited with StarMaker : 26227
Ringtones for iPhone & Ringtone Maker : 25403
Musi - Unlimited Music For YouTube : 25193
AutoRap by Smule : 18202
Spinrilla - Mixtapes For Free : 15053
Napster - Top Music & Radio : 14268
edjing Mix:DJ turntable to remix and scratch music : 13580
Free Music - MP3 Streamer & Playlist Manager Pro : 13443
Free Piano app by Yokee : 13016
Google Play Music : 10118
Certified Mixtapes - Hip H

The Music category is similar to Social Networking and Navigation - there are a few huge apps. This genre doesn't seem to have big potential for newly-created app.

In [25]:
print('Weather\n')
print(f'Name of application: its number of ratings\n')      
for entry in apple_apps:
    if entry[11] == 'Weather':
        print(f'{entry[1]} : {entry[5]}')

Weather

Name of application: its number of ratings

The Weather Channel: Forecast, Radar & Alerts : 495626
The Weather Channel App for iPad – best local forecast, radar map, and storm tracking : 208648
WeatherBug - Local Weather, Radar, Maps, Alerts : 188583
MyRadar NOAA Weather Radar Forecast : 150158
AccuWeather - Weather for Life : 144214
Yahoo Weather : 112603
Weather Underground: Custom Forecast & Local Radar : 49192
NOAA Weather Radar - Weather Forecast & HD Radar : 45696
Weather Live Free - Weather Forecast & Alerts : 35702
Storm Radar : 22792
QuakeFeed Earthquake Map, Alerts, and News : 6081
Moji Weather - Free Weather Forecast : 2333
Hurricane by American Red Cross : 1158
Forecast Bar : 375
Hurricane Tracker WESH 2 Orlando, Central Florida : 203
FEMA : 128
iWeather - World weather forecast : 80
Weather - Radar - Storm with Morecast App : 78
Yurekuru Call : 53
Weather & Radar : 37
WRAL Weather Alert : 25
Météo-France : 24
JaxReady : 22
Freddy the Frogcaster's Weather Station :

From the names of the applications above, their categorization becomes clear - there are applications about the weather in general, and there are those specializing in certain weather conditions.

It's safe to assume that users don't spend much time on the weather app; but, most likely, they do it regularly if the application has already been downloaded - after all, in order to find out the air temperature once a month, you can use the browser.
At first glance, I want to analyze this niche further, but still, the "Reference" category seems more promising, and perhaps it is worth spending more effort on it.
In addition, the next popular category is `Books`, which also correlates with the category `Reference`.

The rest of applications have less than 50,000 reviews on average, let's leave them for now.

# Google Play genre with most users

Let's move on to the analysis of Google applications - everything is a little simpler here, since there is `Installs` column.
For displaying installs we can use **display_freq_table** function.

In [26]:
google_installs = display_freq_table(google_apps, 5)

Amount of categories: 8864
Amount of unique categories: 21

Category : percantage of users

1,000,000+ : 15.73
100,000+ : 11.55
10,000,000+ : 10.55
10,000+ : 10.2
1,000+ : 8.39
100+ : 6.92
5,000,000+ : 6.83
500,000+ : 5.56
50,000+ : 4.77
5,000+ : 4.51
10+ : 3.54
500+ : 3.25
50,000,000+ : 2.3
100,000,000+ : 2.13
50+ : 1.92
5+ : 0.79
1+ : 0.51
500,000,000+ : 0.27
1,000,000,000+ : 0.23
0+ : 0.05
0 : 0.01


We could see here several categories instead of just precise number of installs for each app.  
When calculating the average number of installs for each genre, the result can be blurry, - but we have no choice and let's try it anyway and see what we will get.  
At first we need to remove unwanted characters from string with `Installs`.

In [27]:
for entry in google_apps:
    entry[5] = float(entry[5].replace(',', '').replace('+', ''))

for entry in google_apps[:5]:
    print(entry)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', 10000.0, 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', 5000000.0, 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']
['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', 50000000.0, 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']
['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', 100000.0, 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']
['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', 50000.0, 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


Everithing seems fine, floats are on six position (or position with index 5).
Now we can use the function **avg_count_by_genre** again on Google data set, since installs are now a float and we could sum them up and find average for each genre.

In [28]:
google_popular_genre = avg_count_by_genre(google_apps, 1, 5)

Category : average amount of users/installs

COMMUNICATION : 38456119.17
VIDEO_PLAYERS : 24727872.45
SOCIAL : 23253652.13
PHOTOGRAPHY : 17840110.4
PRODUCTIVITY : 16787331.34
GAME : 15588015.6
TRAVEL_AND_LOCAL : 13984077.71
ENTERTAINMENT : 11640705.88
TOOLS : 10801391.3
NEWS_AND_MAGAZINES : 9549178.47
BOOKS_AND_REFERENCE : 8767811.89
SHOPPING : 7036877.31
PERSONALIZATION : 5201482.61
WEATHER : 5074486.2
HEALTH_AND_FITNESS : 4188821.99
MAPS_AND_NAVIGATION : 4056941.77
FAMILY : 3695641.82
SPORTS : 3638640.14
ART_AND_DESIGN : 1986335.09
FOOD_AND_DRINK : 1924897.74
EDUCATION : 1833495.15
BUSINESS : 1712290.15
LIFESTYLE : 1437816.27
FINANCE : 1387692.48
HOUSE_AND_HOME : 1331540.56
DATING : 854028.83
COMICS : 817657.27
AUTO_AND_VEHICLES : 647317.82
LIBRARIES_AND_DEMO : 638503.73
PARENTING : 542603.62
BEAUTY : 513151.89
EVENTS : 253542.22
MEDICAL : 120550.62


Let's take a closer look at the first few categories: `COMMUNICATION`, `VIDEO_PLAYERS`, `SOCIAL`, `PHOTOGRAPHY`, and we are taking only those apps, which number of installs more than 10 million.

In [29]:
print('COMMUNICATION\n')
print(f'Name of application: its number of installs\n')      
for entry in google_apps:
    if entry[1] == 'COMMUNICATION' and (entry[5] > 10000000):
        print(f'{entry[0]} : {entry[5]}')

COMMUNICATION

Name of application: its number of installs

WhatsApp Messenger : 1000000000.0
imo beta free calls and text : 100000000.0
Contacts : 50000000.0
Android Messages : 100000000.0
Google Duo - High Quality Video Calls : 500000000.0
Messenger – Text and Video Chat for Free : 1000000000.0
imo free video calls and chat : 500000000.0
free video calls and chat : 50000000.0
Skype - free IM & video calls : 1000000000.0
Who : 100000000.0
GO SMS Pro - Messenger, Free Themes, Emoji : 100000000.0
LINE: Free Calls & Messages : 500000000.0
Google Chrome: Fast & Secure : 1000000000.0
Firefox Browser fast & private : 100000000.0
Dolphin Browser - Fast, Private & Adblock🐬 : 50000000.0
UC Browser - Fast Download Private & Secure : 500000000.0
Gmail : 1000000000.0
Mail.Ru - Email App : 50000000.0
Hangouts : 1000000000.0
Azar : 50000000.0
Messenger Lite: Free Calls & Messages : 100000000.0
Kik : 100000000.0
KakaoTalk: Free Calls & Text : 100000000.0
Opera Mini - fast web browser : 100000000.0
O

At a quick glance, we again see a familiar picture: there are several large players in the market, and they account for the largest number of installations.  
It is worth continuing the search.

In [30]:
print('VIDEO_PLAYERS\n')
print(f'Name of application: its number of installs\n')      
for entry in google_apps:
    if entry[1] == 'VIDEO_PLAYERS' and (entry[5] > 10000000):
        print(f'{entry[0]} : {entry[5]}')

VIDEO_PLAYERS

Name of application: its number of installs

YouTube : 1000000000.0
Motorola Gallery : 100000000.0
VLC for Android : 100000000.0
Vote for : 50000000.0
Vigo Video : 50000000.0
Google Play Movies & TV : 1000000000.0
MiniMovie - Free Video and Slideshow Editor : 50000000.0
Samsung Video Library : 50000000.0
LIKE – Magic Video Maker & Community : 50000000.0
MX Player : 500000000.0
Dubsmash : 100000000.0
DU Recorder – Screen Recorder, Video Editor, Live : 50000000.0
KineMaster – Pro Video Editor : 50000000.0
VMate : 50000000.0
HD Video Downloader : 2018 Best video mate : 50000000.0
VivaVideo - Video Editor & Photo Movie : 100000000.0
VideoShow-Video Editor, Video Maker, Beauty Camera : 100000000.0
Ringdroid : 50000000.0
Motorola FM Radio : 100000000.0


Looks like the most apps of this category involves storing large amount of data - this could be an expensive process.  
But there are some popular apps suitable for our search - `video or slideshow editor`.
But if we choose this genre we will need to understand how many such applications exist, as well as what their specific purpose is.  
This is necessary in order to understand what this niche lacks - if the applications overlap in value, the user will rather choose a verified one than a recently created one, and, possibly, will not see our application at all in the search.

In [31]:
print('SOCIAL\n')
print(f'Name of application: its number of installs\n')      
for entry in google_apps:
    if entry[1] == 'SOCIAL' and (entry[5] > 10000000):
        print(f'{entry[0]} : {entry[5]}')

SOCIAL

Name of application: its number of installs

Facebook : 1000000000.0
Facebook Lite : 500000000.0
Tumblr : 100000000.0
Pinterest : 100000000.0
Google+ : 1000000000.0
Badoo - Free Chat & Dating App : 100000000.0
Tango - Live Video Broadcast : 100000000.0
ooVoo Video Calls, Messaging & Stories : 50000000.0
Instagram : 1000000000.0
Snapchat : 500000000.0
MeetMe: Chat & Meet New People : 50000000.0
LinkedIn : 100000000.0
Zello PTT Walkie Talkie : 50000000.0
POF Free Dating App : 50000000.0
SKOUT - Meet, Chat, Go Live : 50000000.0
Tik Tok - including musical.ly : 100000000.0
BIGO LIVE - Live Stream : 100000000.0
VK : 100000000.0


Genre, similar to "COMMUNICATION", no suggestions what we can add to this category.

In [32]:
print('PHOTOGRAPHY\n')
print(f'Name of application: its number of installs\n')      
for entry in google_apps:
    if entry[1] == 'PHOTOGRAPHY' and (entry[5] > 10000000):
        print(f'{entry[0]} : {entry[5]}')

PHOTOGRAPHY

Name of application: its number of installs

Motorola Camera : 50000000.0
B612 - Beauty & Filter Camera : 100000000.0
InstaBeauty -Makeup Selfie Cam : 50000000.0
Selfie Camera - Photo Editor & Filter & Sticker : 50000000.0
YouCam Makeup - Magic Selfie Makeovers : 100000000.0
ASUS Gallery : 50000000.0
Sweet Selfie - selfie camera, beauty cam, photo edit : 100000000.0
Google Photos : 1000000000.0
Square InPic - Photo Editor & Collage Maker : 50000000.0
Retrica : 100000000.0
VSCO : 50000000.0
PhotoWonder: Pro Beauty Photo Editor Collage Maker : 50000000.0
Photo Effects Pro : 50000000.0
Photo Editor Selfie Camera Filter & Mirror Image : 50000000.0
Photo Editor Pro : 100000000.0
Pic Collage - Photo Editor : 50000000.0
Photo Editor by Aviary : 50000000.0
Video Editor Music,Cut,No Crop : 50000000.0
Pixlr – Free Photo Editor : 50000000.0
Adobe Photoshop Express:Photo Editor Collage Maker : 50000000.0
BeautyPlus - Easy Photo Editor & Selfie Camera : 100000000.0
PicsArt Photo Studio

As with the "VIDEO_PLAYERS" genre, `PHOTOGRAPHY` also has some potential - it can be another photo editor for certain purposes. But there are many such applications, it is worth exploring this area better in the future.

In [33]:
print('PRODUCTIVITY\n')
print(f'Name of application: its number of installs\n')      
for entry in google_apps:
    if entry[1] == 'PRODUCTIVITY' and (entry[5] > 1000000):
        print(f'{entry[0]} : {entry[5]}')

PRODUCTIVITY

Name of application: its number of installs

Microsoft Word : 500000000.0
All-In-One Toolbox: Cleaner, Booster, App Manager : 10000000.0
AVG Cleaner – Speed, Battery & Memory Booster : 10000000.0
QR Scanner & Barcode Scanner 2018 : 10000000.0
Chrome Beta : 10000000.0
Microsoft Outlook : 100000000.0
Google PDF Viewer : 10000000.0
My Claro Peru : 5000000.0
Google Assistant : 10000000.0
Microsoft OneDrive : 100000000.0
Calculator - unit converter : 50000000.0
Microsoft OneNote : 100000000.0
Metro name iD : 10000000.0
Google Keep : 100000000.0
Archos File Manager : 5000000.0
ES File Explorer File Manager : 100000000.0
ASUS SuperNote : 10000000.0
HTC File Manager : 10000000.0
Dropbox : 500000000.0
ASUS Quick Memo : 10000000.0
HTC Calendar : 10000000.0
Google Docs : 100000000.0
ASUS Calling Screen : 10000000.0
lifebox : 5000000.0
Yandex.Disk : 5000000.0
Content Transfer : 5000000.0
HTC Mail : 10000000.0
Advanced Task Killer : 50000000.0
MyVodafone (India) - Online Recharge & Pa

We could see here a lot of useful tools for working with computer, this area definitely has some room for imagination.  
But also it is quiet broad - the search for the market needs of this area will take some time, especially for two markets. And, in addition, there is nothing similar in App Store in the first positions - it will be hard to create something useful for both stores.

Our task now is to `find such an application genre that will be relevant for both markets - App Store and Google Play`.  
So, let's take a look at `BOOKS_AND_REFERENCE` Google Play genre (8.767.811 installs, 11th place) - because we found this genre as suitable for App Store.  
And then we will come back to App Store genre `Photo & Video` (ratings count: 28.441, 9th place), because in Google Play genres `PHOTOGRAPHY` and `VIDEO_PLAYERS` we found promising.

In [34]:
print('BOOKS_AND_REFERENCE\n')
print(f'Name of application: its number of installs\n')      
for entry in google_apps:
    if entry[1] == 'BOOKS_AND_REFERENCE' and (entry[5] > 1000000):
        print(f'{entry[0]} : {entry[5]}')

BOOKS_AND_REFERENCE

Name of application: its number of installs

Wikipedia : 10000000.0
Cool Reader : 10000000.0
FBReader: Favorite Book Reader : 10000000.0
Google Play Books : 1000000000.0
AlReader -any text book reader : 5000000.0
Ebook Reader : 5000000.0
Read books online : 5000000.0
Ancestry : 5000000.0
HTC Help : 10000000.0
Moon+ Reader : 10000000.0
Bible : 100000000.0
Amazon Kindle : 100000000.0
Aldiko Book Reader : 10000000.0
Wattpad 📖 Free Books : 100000000.0
Dictionary - WordWeb : 5000000.0
50000 Free eBooks & Free AudioBooks : 5000000.0
Al-Quran (Free) : 10000000.0
Al Quran Indonesia : 10000000.0
Al'Quran Bahasa Indonesia : 10000000.0
Al Quran : EAlim - Translations & MP3 Offline : 5000000.0
Quran for Android : 10000000.0
Audiobooks from Audible : 100000000.0
Dictionary.com: Find Definitions for English Words : 10000000.0
English Dictionary - Offline : 10000000.0
Bible KJV : 5000000.0
NOOK: Read eBooks & Magazines : 10000000.0
Dictionary : 10000000.0
Spanish English Translat

Just like App Store, `BOOKS_AND_REFERENCE` genre in Google Play has
religious books, dictionaries, electronic readers and so on.

In [35]:
print('Photo & Video\n')
print(f'Name of application: its number of installs\n')      
for entry in apple_apps:
    if entry[11] == 'Photo & Video':
        print(f'{entry[1]} : {entry[5]}')

Photo & Video

Name of application: its number of installs

Instagram : 2161558
Snapchat : 323905
YouTube - Watch Videos, Music, and Live Streams : 278166
Pic Collage - Picture Editor & Photo Collage Maker : 123433
Funimate video editor: add cool effects to videos : 123268
musical.ly - your video social network : 105429
Photo Collage Maker & Photo Editor - Live Collage : 93781
Vine Camera : 90355
Google Photos - unlimited photo and video storage : 88742
Flipagram : 79905
Mixgram - Picture Collage Maker - Pic Photo Editor : 54282
Shutterfly: Prints, Photo Books, Cards Made Easy : 51427
Pic Jointer – Photo Collage, Camera Effects Editor : 51330
Color Pop Effects - Photo Editor & Picture Editing : 45320
Photo Grid - photo collage maker & photo editor : 40531
iSwap Faces LITE : 39722
MOLDIV - Photo Editor, Collage & Beauty Camera : 39501
Photo Editor by Aviary : 39501
Photo Lab: Picture Editor, effects & fun face app : 34585
Rookie Cam - Photo Editor & Filter Camera : 33921
FotoRus -Camera

Despite the fact that the dominant part of the users is again in the hands of well-known applications, here we can still see `photo and video editing applications`, which also have their own audience.

Now that we've outlined the common genres for both markets, let's take a quick look at Google Play's secondary genres - perhaps it will also provide some insight.

In [36]:
google_avg_genres_minor = avg_count_by_genre(google_apps, 9, 5)

Category : average amount of users/installs

Communication : 38456119.17
Adventure;Action & Adventure : 35333333.33
Video Players & Editors : 24947335.8
Social : 23253652.13
Arcade : 22888365.49
Casual : 19569221.6
Puzzle;Action & Adventure : 18366666.67
Photography : 17840110.4
Educational;Action & Adventure : 17016666.67
Productivity : 16787331.34
Racing : 15910645.68
Travel & Local : 14051476.15
Casual;Action & Adventure : 12916666.67
Action : 12603588.87
Strategy : 11199902.53
Tools : 10802461.25
Lifestyle;Pretend Play : 10000000.0
Casual;Music & Video : 10000000.0
Tools;Education : 10000000.0
Card;Action & Adventure : 10000000.0
Adventure;Education : 10000000.0
Role Playing;Brain Games : 10000000.0
News & Magazines : 9549178.47
Music : 9445583.33
Educational;Pretend Play : 9375000.0
Puzzle;Brain Games : 9280666.67
Word : 9094458.7
Racing;Action & Adventure : 8816666.67
Books & Reference : 8767811.89
Puzzle : 8302861.91
Video Players & Editors;Music & Video : 7500000.0
Shopping : 7

Minor Google genres confirm the prevailing opinion - in the first places there are `Communication`, `Video Players & Editors`.  
And also some "Action & Adventure" and "Arcade", which after quick search are recognized as games genres, which are not so popular in App Store.

Worth reminding the most popular App Store genres, no games or adventures here (by rating count):

Navigation : 86090.33  
Reference : 74942.11  
Social Networking : 71548.35  
Music : 57326.53  
Weather : 52279.89  
Book : 39758.5  
Food & Drink : 33333.92  
Finance : 31467.94  

The last step is to `correlate the rating` of applications in stores `by their number and by their popularity`.

App Store
Category : percantage of users

Games : 58.14  
Entertainment : 7.89  
Photo & Video : 4.97  
Education : 3.66  
Social Networking : 3.29  
Shopping : 2.61  
Utilities : 2.52  
Sports : 2.14  
Music : 2.05  
Health & Fitness : 2.02  
Productivity : 1.74  
Lifestyle : 1.58  
News : 1.34  
Travel : 1.24  
Finance : 1.12  
Weather : 0.87  
Food & Drink : 0.81  
Reference : 0.56  
Business : 0.53  
Book : 0.43  
Navigation : 0.19  
Medical : 0.19  
Catalogs : 0.12  

As we could see, the total volume of applications of the genres we have chosen - "Photo & Video" (4.97), "Reference" (0.56), "Book" (0.43), - is small in App Store.

Google Play, major genres
Category : percantage of users

FAMILY : 18.91  
GAME : 9.72  
TOOLS : 8.46  
BUSINESS : 4.59  
LIFESTYLE : 3.9  
PRODUCTIVITY : 3.89  
FINANCE : 3.7  
MEDICAL : 3.53  
SPORTS : 3.4  
PERSONALIZATION : 3.32  
COMMUNICATION : 3.24  
HEALTH_AND_FITNESS : 3.08  
PHOTOGRAPHY : 2.94  
NEWS_AND_MAGAZINES : 2.8  
SOCIAL : 2.66  
TRAVEL_AND_LOCAL : 2.34  
SHOPPING : 2.25  
BOOKS_AND_REFERENCE : 2.14  
DATING : 1.86  
VIDEO_PLAYERS : 1.79  
MAPS_AND_NAVIGATION : 1.4  
FOOD_AND_DRINK : 1.24  
EDUCATION : 1.16  
ENTERTAINMENT : 0.96  
LIBRARIES_AND_DEMO : 0.94  
AUTO_AND_VEHICLES : 0.93  
HOUSE_AND_HOME : 0.82  
WEATHER : 0.8  
EVENTS : 0.71  
PARENTING : 0.65  
ART_AND_DESIGN : 0.64  
COMICS : 0.62  
BEAUTY : 0.6

On Google Play genres "COMMUNICATION" (3.24 %), "SOCIAL" (2.66 %), "BOOKS_AND_REFERENCE" (2.14 %), and "VIDEO_PLAYERS" (1.79 %) - also occupy a small part of all apps.

This means that we will have potential space in both markets for our ideas if we choose the genres above.

# Results

Of course, there is still a lot of work ahead - it is necessary to analyze the needs of the markets, as well as to determine the portrait and features of our future application.  
But it is already possible to identify several genres of applications that are popular among users and have a certain potential for creating an application.

The first direction is `References & Books`.  
Here an application with interactive tools looks promising - for a fiction book, or better - for a series of books, or for educational materials on any subject.

The second direction is `Photo & Video`.  
This environment seems to be very fertile for applications - different photo and video editors can serve slightly different purposes, hence their diversity.

On the whole, we should also highlight all-favourite `Social Networking` genre, also known as `Social` or `Communications`.  
But here the analysis of market needs will not be enough - good ideas are required, otherwise the new application may not be noticed. In addition, to be competitive, an application of this genre will require significant marketing costs.

Summing up, we can say that it is quite realistic to create an application that can be profitable for both markets - Apple Store and Google Play, and maybe our app won't be extremely popular, but it has the chances to be balanced in terms of money, efforts and time.